<a id='Homework'></a>
# Homework

Theory (5 points):
- Complete theory questions in Google Form
- Take a look at all the links
- Read and analyze all theory `TODO`s

Practice (10 points):
1. Take 2-3 channels from `KyivChannels_Dataset_v01`
2. Apply Clustering OR/AND Topic Modelling techniques to find topics of these channels. Ideal output: `channel_name:[topic_1, topic_2, topic_3]`. Examples: `Крипта Миколи : [криптовалюта, біржа]`
3. (Advanced) Try to come up with a universal approach
4. (Advanced) Apply your approach on other channels

Here is a list of standard topics for TG channels (from TGStat). In the best-case scenario use them as topics.
```
Adult
Art
Blogs
Bookmaking
Books
Business and startups
Career
Courses and guides
Cryptocurrencies
Darknet
Design
Economics
Education
Edutainment
Erotic
Esoterics
Family & Children
Fashion and beauty
Food and cooking
Games
Handiwork
Health and Fitness
Humor and entertainment
Instagram
Interior and construction
Law
Linguistics
Marketing, PR, advertising
Medicine
Music
Nature
News and media
Other
Pictures and photos
Politics
Psychology
Quotes
Religion
Sales
Shock content
Software & Applications
Sport
Technologies
Telegram
Transport
Travel
Video and films
```